## Runnning with gammapy-dev/IRF_model
Fitting asimov datasets with nuisance parameters based on the different livetimes

### Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from gammapy.maps import Map
from astropy.coordinates import SkyCoord, Angle
from gammapy.modeling import Fit,  Parameters, Covariance , Parameter
from gammapy.datasets import MapDataset ,Datasets, FluxPointsDataset
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
    PointSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    SpatialModel,
    FoVBackgroundModel,
    PiecewiseNormSpectralModel,
)
from gammapy.estimators import TSMapEstimator, ExcessMapEstimator
from gammapy.estimators import FluxPoints, FluxPointsEstimator
from scipy.interpolate import interp2d

from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys
sys.path.append('../')
import Dataset_load 

from  Dataset_Setup import Setup, GaussianCovariance_matrix

from matplotlib import rc
rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
rc("text", usetex=True)


## Read dataset

In [ ]:
c = Dataset_load.load_config()
awo, aw, ewo, ew = c['_colors']  
c

In [ ]:
livetime = c['livetime']
zero = c['zero'] 
norm = c['norm'] 
tilt = c['tilt'] 
bias = c['bias'] 
resolution = c['resolution'] 
magnitude = c['magnitude'] 
corrlength = c['corrlength']
sys = c['sys'] 
folder = c['folder']
parameter_names = c['parameter_names']        
nbidx = 0
print(livetime)
print(sys)

In [ ]:
c['model']

In [ ]:
%%time
amplitude= Parameter('amplitude', value = 3.85e-12, unit=u.Unit("1 / (TeV s cm2)"))
dataset_input_point  = Dataset_load.create_asimov(model = c['model'], source = c['source'], 
                                               livetime = f"{livetime}-hr",
                                        parameters = [amplitude],
                                           spatial_model =None)
from gammapy.modeling.models import GaussianSpatialModel
gaussian = GaussianSpatialModel(lon_0 = dataset_input_point.models.parameters['lon_0'],
                                lat_0 = dataset_input_point.models.parameters['lat_0'],
                                sigma = 0.3 *u.deg
                               )


dataset_input  = Dataset_load.create_asimov(model = c['model'], source = c['source'], 
                                               livetime = f"{livetime}-hr",
                                        parameters = [amplitude],
                                           spatial_model =gaussian)


In [ ]:
dataset_input.npred().sum_over_axes().plot(add_cbar =1)

In [ ]:
    
setup = Setup(dataset_input=dataset_input)
#setup.set_up_irf_sys(bias, resolution, norm, tilt)
dataset_asimov, dataset_asimov_N = setup.run()
# irf model
#setup.set_irf_model(dataset_asimov_N)
if "Bkg" in sys or "BKG" in sys:

    # piece wise model
    # remove old bkg model
    setup.set_up_bkg_sys_V( breake = 10,
                        index1 = 2,
                        index2 = 1.5, 
                        magnitude = magnitude )

    dataset_asimov, dataset_asimov_N = setup.run()

    #setup.unset_model(dataset_asimov_N, FoVBackgroundModel)
    #setup.set_piecewise_bkg_model(dataset_asimov_N)
    # energy of the following parameters smaller than ethrshold
    dataset_asimov_N.background_model.parameters['norm0'].frozen = True
    dataset_asimov_N.background_model.parameters['norm1'].frozen = True
    dataset_asimov_N.background_model.parameters['norm2'].frozen = True
    dataset_asimov_N.background_model.parameters['norm3'].frozen = True
    #setup.set_bkg_prior(dataset_asimov_N, magnitude, corrlength)
    

In [ ]:
print(dataset_asimov)

In [ ]:
print(dataset_asimov_N)

In [ ]:
print(setup.dataset_helper)

In [ ]:
setup.dataset_helper.counts.sum_over_axes().plot(add_cbar = 1)

In [ ]:
setup.dataset_helper.plot_residuals()

In [ ]:
dataset_asimov_N.counts.sum_over_axes().plot(add_cbar = 1)

In [ ]:
dataset_asimov_N.plot_residuals()

In [ ]:
%%time
fitting = 0
if fitting:
    fit_cor = Fit(store_trace=0)
    minuit_opts = {"tol": 0.1, "strategy": 2}
    fit_cor.backend = "minuit"
    fit_cor.optimize_opts = minuit_opts
    result_cor = fit_cor.run(dataset_asimov)
    print(result_cor)
    print("saving")
    path = f'../{folder}/data/0_model_livetime_{livetime}.yml'
    dataset_asimov.models.write(path,overwrite=True)
    
    
else:
    path = f'../{folder}/data/0_model_livetime_{livetime}.yml'
    dataset_asimov.models = Models.read(path)
    print(path[:-4])
    cov = dataset_asimov.models.read_covariance(path = path[:-4]+"_covariance.dat", filename = "",
                                            format="ascii.fixed_width")


In [ ]:
print(dataset_asimov_N)

In [ ]:
%%time
fitting = 1
if fitting:
    fit_cor = Fit(store_trace=0)
    result_cor = fit_cor.run(dataset_asimov_N)
    print(result_cor)
    print("saving")
    path = f'../{folder}/data/0_model_nui_livetime_{livetime}.yml'
    dataset_asimov_N.models.write(path,overwrite=True)
    
    
else:
    path = f'../{folder}/data/0_model_nui_livetime_{livetime}.yml'
    dataset_asimov_N.models = Models.read(path)


In [ ]:
print(dataset_asimov_N.background_model)

In [ ]:
ax = dataset_asimov_N.background_model.spectral_model.plot()
ax.set_yscale("linear")
print(dataset_asimov_N.background_model.spectral_model.parameters.value)

In [ ]:
from Dataset_Setup import GaussianCovariance_matrix
cov = GaussianCovariance_matrix(size = len(setup.emask()),
                                        magnitude = magnitude, 
                                        corrlength = corrlength)
plt.imshow(cov.cov())
print(cov.cov()[-1])

In [ ]:
plt.imshow(dataset_asimov_N.background_model.spectral_model.parameters.prior[1].covariance_matrix)

In [ ]:
dataset_asimov_N.background_model.spectral_model.parameters.value  = setup.dataset_helper.background_model.spectral_model.parameters.value



In [ ]:
dataset_asimov_N.background_model.spectral_model

In [ ]:
ax = dataset_asimov.plot_residuals_spectral(color = 'tab:orange')
dataset_asimov_N.plot_residuals_spectral(ax = ax, color = 'tab:blue')

In [ ]:

ax = dataset_input.models[0].spectral_model.plot(color = 'black', label = 'Input')
dataset_asimov_N.models[0].spectral_model.plot(color = 'tab:blue', linestyle = 'dashed',
                                                 label = "With nui.")
dataset_asimov_N.models[0].spectral_model.plot_error(ax = ax, facecolor = 'tab:blue')

dataset_asimov.models[0].spectral_model.plot(color = 'tab:orange', linestyle = 'dashed',
                                                 label = "Without nui.")
dataset_asimov.models[0].spectral_model.plot_error(ax = ax, facecolor = 'tab:orange')



ax.legend()

In [ ]:
print(dataset_asimov.models[0])

In [ ]:


ax = dataset_asimov_N.background_model.spectral_model.plot()
setup.dataset_helper.background_model.spectral_model.plot()
ax.set_yscale("linear")
print(dataset_asimov_N.background_model.spectral_model.parameters.value)

In [ ]:
setup.dataset_helper.background_model